External Libraries

In [6]:
from collections import deque
import numpy as np
import tweepy
import json
import pandas as pd
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
from collections import deque
import random

Created Packages

In [7]:
import CentroidUtils as cf
import FeaturesSelectorNew as fs
import TextPreprocessor as tp
import ImpliedPreprocessor as ip

Global Variables

In [8]:
corpus=[]
ner_corpus=[]
list_of_tweetobjects=[]
list_of_active_centroids=[]
queue_of_current_centroids = deque()
final_topics=[]

In [9]:
df2 = pd.read_csv("test.csv")

Intialisation phase

In [10]:
list_of_intial_tweets = df2["text"][1:10]
for tweet in list_of_intial_tweets:
    tweet = tp.clean(tweet)
    tweet = ip.imply_preprocess(tweet)
    #print(tweet)
    corpus.append(tweet)

In [11]:
ner_corpus,corpus=fs.initialisation(list_of_intial_tweets)

Functions

In [19]:
def descend(timer):
    final_topics=[]
    stored_timelimit = timer
    while(timer):
        descend_centroids = queue_of_current_centroids.popleft()
        #print(descend_centroids.cluster_label)
        list_of_tweetobject = descend_centroids.buffer    #change
        dynamic_similarity_measure=10**(descend_centroids.height)*0.002
        dynamic_buffer_count=(0.125)**(descend_centroids.height)*5
        print("dynamic_similarity_measure",dynamic_similarity_measure)
        print("dynamic_buffer_count",dynamic_buffer_count)
        list_of_active_centroid = cf.give_list_of_active_clusters(list_of_tweetobject,dynamic_similarity_measure,dynamic_buffer_count)
        if(len(list_of_active_centroid)==0):
            #descend_centroids.buffer.clear()
            if descend_centroids.childArray :
                [queue_of_current_centroids.append(i) for i in descend_centroids.childArray]
            else :
                final_topics.append(descend_centroids)
        else:
            descend_centroids.buffer.clear()
            #print(len(list_of_active_centroids))
            children_of_descend_centroids = descend_centroids.childArray
            if children_of_descend_centroids :
                for new_child in list_of_active_centroid:
                    max_similarity = -1
                    most_similar_old_child =None
                    for old_child in children_of_descend_centroids:
                        current_similarity = cf.similarityBetweenCentroids(old_child,new_child)
                        if(current_similarity>max_similarity):
                            max_similarity = current_similarity
                            most_similar_old_child = old_child
                    if most_similar_old_child not in None : 
                        if(max_similarity>0.5):
                            cf.mergeClusters(most_similar_old_child,new_child,descend_centroids)
                        else:
                            cf.addChildCluster(descend_centroids,new_child)
                            queue_of_current_centroids.append(child_centroid)
                    else :
                        cf.addChildCluster(descend_centroids,new_child) 
                        queue_of_current_centroids.append(child_centroid)
            else :
                for child_centroid in list_of_active_centroid:
                    cf.addChildCluster(descend_centroids,child_centroid)
                    queue_of_current_centroids.append(child_centroid)
        timer=timer-1
        #theta - deffering 
        #cf.bufferpruning()
        
        
    print("No of Topics Detected : " , len(final_topics) + len(queue_of_current_centroids))
    #print("--------------------------------------------")
    cf.giveDetails(final_topics)
    cf.giveDetails(queue_of_current_centroids)
   # print("--------------------------------------------")
    print("without descending")
    print(len(list_of_active_centroids))
    cf.giveDetails(list_of_active_centroids)

Execution

In [20]:
allow_interrupt = True 
list_of_tweets=[]
readAt = 1001 
count=1 
while True:
    try: 
        x = random.randint(0,1)
        import time
        time.sleep(int(x)) 
        tweet_data = df2['text'][readAt] 
        readAt = readAt+1
        print("--------------Tweet count #",count,"---------------------------------------")
        print("Tweet:::::::",tweet_data)
        tweet = tp.clean(tweet_data) 
        tweet = ip.imply_preprocess(tweet) 
        tweet_keywords,corpus,ner= fs.give_top_utility_score_features(tweet,corpus, ner_corpus) 
        new_tweet = cf.makeTweetObjects(tweet_keywords) 
        list_of_tweetobjects.append(new_tweet) 
        print("keywords:::::::",tweet_keywords) 
        count=count+1 
        print("------------------------------------------------------") 
    except KeyboardInterrupt: 
        if allow_interrupt: 
            list_of_active_centroids = cf.give_list_of_active_clusters(list_of_tweetobjects,0.002,5)
            #print("length of active centroids",len(list_of_active_centroids))
            cf.giveDetails(list_of_active_centroids) 
            refine_time = int(input("start proccessing after how many seconds ??? ")) 
            
            if refine_time == -1 : 
                print ('interrupted!') 
                break 
            else: 
                queue_of_current_centroids = deque() 
                [queue_of_current_centroids.append(i) for i in list_of_active_centroids] 
                if len(queue_of_current_centroids)!=0:
                    
                    print("*******************************Analysis*****************************************")
                    descend(refine_time) 
                    print("********************************************************************************")
                else :
                    print("No active clusters!!!!!!!!")
                pass

--------------Tweet count # 1 ---------------------------------------
Tweet::::::: RT @kilmeade: #VPDebate wow this is a huge leap @timkaine links trump taxes to 911? @realDonaldTrump ..Sorus, buffet  all had big losses an…
keywords::::::: ['buffet loss', 'link trump', 'trump tax', 'tax Sorus', 'Sorus buffet']
------------------------------------------------------
--------------Tweet count # 2 ---------------------------------------
Tweet::::::: RT @benensonj: Pence just lied through his teeth when he denied Trump said more countries should get nukes!
keywords::::::: ['country nuke', 'said country', 'teeth denied', 'denied Trump', 'Trump said', 'Trump']
------------------------------------------------------
--------------Tweet count # 3 ---------------------------------------
Tweet::::::: RT @chrislhayes: This is hilarious. Mike Pence shaking his head as Tim Kaine lists Donald Trump insults as if it's all made up or that Kain…
keywords::::::: ['shaking head', 'head Kaine', 'Kaine list'

keywords::::::: ['trump leave', 'leave pay', 'pay tax', 'tax talk', 'defense trump']
------------------------------------------------------
--------------Tweet count # 23 ---------------------------------------
Tweet::::::: RT @ezraklein: It sort of works in the debate, but Pence shaking head, saying "no he hasn't" is going to look bad in ads next to Trump sayi…
keywords::::::: ['going look', 'look next', 'next Trump', 'Trump sayi…', 'head saying', 'Trump']
------------------------------------------------------
--------------Tweet count # 24 ---------------------------------------
Tweet::::::: RT @TerryGlavin: 1. Pence cleaned the floor with Kaine on Syria. Until his last line about how Trump's the one to face Putin. Comical. #VPD…
keywords::::::: ['cleaned floor', 'floor Kaine', 'Kaine Syria', 'Syria last', 'last line', 'Kaine', 'Syria', 'Putin', 'Comical']
------------------------------------------------------
--------------Tweet count # 25 ---------------------------------------
Twe

keywords::::::: ['based piece', 'piece hav…', 'supposed based', 'youre supposed', 'code youre', 'Pence', 'Trump']
------------------------------------------------------
--------------Tweet count # 45 ---------------------------------------
Tweet::::::: RT @literally_trump: Donald Trump said Putin is a better leader than President Obama. And his running mate agrees. https://t.co/kRD9vG1Wq0
keywords::::::: ['mate agrees', 'running mate', 'better leader', 'leader President', 'President Obama', 'Donald', 'Trump', 'Putin', 'Obama']
------------------------------------------------------
--------------Tweet count # 46 ---------------------------------------
Tweet::::::: @neeratanden @JenGranholm Do they keep Pence in a soundproof room so he doesn't hear what Trump actually says? Or Pence just not listening?
topic is: ['trump', 'tax', 'Mike', 'Pence', 'Trump']
--------------------------
cluster_value 204.23126105920235
topic is: ['said', 'Trump']
--------------------------
cluster_value 321.55

keywords::::::: ['need tell', 'tell cap', 'cap make', 'make lie', 'lie true', 'Trump']
------------------------------------------------------
--------------Tweet count # 59 ---------------------------------------
Tweet::::::: @timkaine Keep the info coming abt Trump Russian connection. Trump not knowing Russians invaded Ukraine &amp; seized Crimea. #VPDebate
keywords::::::: ['info coming', 'coming Trump', 'Trump Russian', 'Russian connection', 'connection Trump', 'Trump', 'Trump', 'Ukraine', 'Crimea']
------------------------------------------------------
--------------Tweet count # 60 ---------------------------------------
Tweet::::::: RT @TheDailyEdge: #VPDebate

Pence: "Donald Trump never said those things!"

Fact checkers: https://t.co/NIcdEbBFUe
keywords::::::: ['said thing', 'thing Fact', 'Fact checker', 'checker', 'Pence Donald', 'Donald', 'Trump']
------------------------------------------------------
--------------Tweet count # 61 ---------------------------------------
Tweet

keywords::::::: ['ticking time', 'time', 'clock still', 'still ticking', 'ticking']
------------------------------------------------------
--------------Tweet count # 81 ---------------------------------------
Tweet::::::: @MorrowKatharine presidents like trump keep immigrants and hard working families from getting to what second generation kids like me have.
keywords::::::: ['second generation', 'generation kid', 'kid like', 'getting second', 'family getting']
------------------------------------------------------
--------------Tweet count # 82 ---------------------------------------
Tweet::::::: RT @KevinMKruse: "I'm very happy to defend Donald Trump," says the man who has just spent 67 minutes finding every way possible to avoid do…
keywords::::::: ['possible avoid', 'minute finding', 'finding every', 'every possible', 'spent minute', 'Donald', 'Trump']
------------------------------------------------------
--------------Tweet count # 83 ---------------------------------------
Tweet

...

--------------Tweet count # 719 ---------------------------------------
Tweet::::::: RT @TheDailyEdge: #VPDebate

Pence: "Donald Trump never said those things!"

Fact checkers: https://t.co/NIcdEbBFUe
keywords::::::: ['said thing', 'thing Fact', 'Fact checker', 'checker', 'Pence Donald', 'Donald', 'Trump']
------------------------------------------------------
--------------Tweet count # 720 ---------------------------------------
Tweet::::::: RT @JustinWolfers: We've now got tons of post-debate polling. Looks like Clinton +4 immediately after the debate, widening to Clinton +6 po…
keywords::::::: ['debate', 'widening', 'Weve ton', 'ton postdebate', 'postdebate polling', 'Clinton', 'Clinton']
------------------------------------------------------
--------------Tweet count # 721 ---------------------------------------
Tweet::::::: RT @PGourevitch: Pence knows many little things that Trump doesn’t know but he doesn’t know one big thing, how to defend Trump’s statements…
keywords::::::: [

keywords::::::: ['probably doesnt', 'doesnt tax', 'insult probably', 'veteran insult', 'troop veteran', 'Mike', 'Pence', 'Trump']
------------------------------------------------------
--------------Tweet count # 742 ---------------------------------------
Tweet::::::: @nickconfessore that's because Trump can't answer them.
keywords::::::: ['answer', 'thats Trump', 'Trump cant', 'cant answer', 'Trump', 'Trump']
------------------------------------------------------
--------------Tweet count # 743 ---------------------------------------
Tweet::::::: RT @jonlovett: Kaine is going to hit back over and over because this isn't about him. Pence won't defend Trump because it IS about him.
keywords::::::: ['wont defend', 'defend Trump', 'Trump', 'Kaine going', 'going back', 'Kaine', 'Trump']
------------------------------------------------------
--------------Tweet count # 744 ---------------------------------------
Tweet::::::: RT @MarkSimoneNY: Kaine's theme in this debate is that the number

--------------Tweet count # 764 ---------------------------------------
Tweet::::::: RT @LOLGOP: It was really rude of Tim Kaine to repeat things Donald Trump said in front of Mike Pence.
keywords::::::: ['Pence', 'really rude', 'rude Kaine', 'Kaine repeat', 'repeat thing', 'Kaine', 'Donald', 'Trump', 'Mike', 'Pence']
------------------------------------------------------
--------------Tweet count # 765 ---------------------------------------
Tweet::::::: RT @StuPolitics: Pence needs to watch videos of Trump speeches.  He might know what Trump has said.
keywords::::::: ['watch video', 'video Trump', 'Trump speech', 'speech might', 'might know', 'Trump', 'Trump']
------------------------------------------------------
--------------Tweet count # 766 ---------------------------------------
Tweet::::::: RT @JustinWolfers: Pence just denied that Trump called NATO obsolete. https://t.co/dAVsniqAE2
keywords::::::: ['obsolete', 'Pence denied', 'denied Trump', 'Trump called', 'called NATO', 'Tr

start proccessing after how many seconds ??? 12
*******************************Analysis*****************************************
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
dynamic_similarity_measure 0.002
dynamic_buffer_count 5.0
No of Topics Detected :  18
topic is: ['going', 'nuclear weapon', 'Trump']
--------------------------
cluster_value 185.3842268842269
topic is: ['think theyre', 'Kaine', 'Trum

start proccessing after how many seconds ??? -1
interrupted!
